In [6]:
import sqlite3
import numpy as np
import pandas as pd
import json
import nltk
from nltk.corpus import words

nltk.download("words")

[nltk_data] Downloading package words to /home/alireza/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

# Word frequency

In [2]:
def read_clean_data():
    # Connect to the SQLite database
    conn = sqlite3.connect("words.db")

    # Initialize an empty list to hold DataFrames
    dfs = []

    for i in range(3, 35):
        query = f'SELECT * FROM "{i}"'
        df = pd.read_sql_query(query, conn)
        df.drop("ID_I", axis=1, inplace=True)
        dfs.append(df)

    # Concatenate all DataFrames in the list
    final_df = pd.concat(dfs, ignore_index=True)
    final_df.columns = ["word", "frequency"]

    # convert frequency to int
    final_df["frequency"] = final_df["frequency"].astype(int)

    # Sort the DataFrame by frequency in descending order
    final_df = final_df.sort_values("frequency", ascending=False).reset_index(drop=True)

    # Close the connection
    conn.close()

    return final_df

In [3]:
all_words = read_clean_data()

# print df size
print(f"df size is {all_words.shape}")

# Display the concatenated DataFrame
print(all_words)

df size is (608943, 2)
           word    frequency
0           the  23135936835
1           and  12997704754
2           for   5933354779
3          that   3400041846
4          this   3228476270
...         ...          ...
608938  unsteek            1
608939  unsteck            1
608940  unstate            1
608941  unstain            1
608942  unstaid            1

[608943 rows x 2 columns]


In [7]:
# Create a set of English words for fast lookup
english_words_set = set(words.words())

# Vectorized approach to check if words are in the English word list
all_words["is_english"] = np.vectorize(lambda x: x.lower() in english_words_set)(
    all_words["word"]
)

In [9]:
df = all_words[(all_words["is_english"] == True)][:50_000].reset_index(drop=True)

print(df)


                word    frequency  is_english
0                the  23135936835        True
1                and  12997704754        True
2                for   5933354779        True
3               that   3400041846        True
4               this   3228476270        True
...              ...          ...         ...
49995  refractometry        13607        True
49996      reconvert        13606        True
49997   planetesimal        13606        True
49998  paravertebral        13604        True
49999          alula        13603        True

[50000 rows x 3 columns]


In [37]:
# write column word to csv without header
df["word"].to_csv("english_words.csv", index=False, header=None)

# GRE/TOEFL/IELTS

In [19]:
# GRE words
js_files = ['warm-up.js', 'intermediate.js', 'hard.js', 'high-frequency-gre.js']

all_js_dfs = []
for js_file in js_files:
    js_df = pd.read_json(f"data/{js_file}")
    all_js_dfs.append(js_df)

data_1 = pd.concat(all_js_dfs, ignore_index=True)
data_1 = data_1["word"].unique()


len(data_1)

781

In [ ]:
def extract_words(file_path, delimiter):
    words = set()  # Use a set to store unique words
    with open(file_path, "r") as file:
        for line in file:
            if delimiter in line:  # Only process lines containing the delimiter
                word = line.split(delimiter)[
                    0
                ].strip()
                words.add(word)  # Add the word to the set (ensures uniqueness)
    return list(words)  # Convert the set back to a list


In [13]:



# Example usage
file_path = "your_file.txt"  # Replace with the path to your file
delimiter = ":"  # The delimiter can be any character, for example ':'
unique_words = extract_words(file_path, delimiter)
print(unique_words)


list